# Build a Customer Support Router Agentic RAG System (Databricks Vector Search)

In this project, we will leverage the power of AI Agents and RAG Systems to build an intelligent Router Agentic RAG System to handle customer support queries using a custom knowledgebase powered by **Databricks Vector Search**.

![](https://i.imgur.com/bLCdxCI.png)

### Intelligent Router Agentic RAG System

This project focuses on building an **Intelligent Router Agentic RAG System** that combines intelligent query analysis, sentiment detection, and dynamic routing with Retrieval-Augmented Generation (RAG) to handle diverse user inquiries efficiently. The workflow includes the following components:

1. **Query Categorization and Sentiment Analysis**:
   - The system uses an LLM to analyze the user's query and determine:
     - **Query Category**: Identifies the type of problem, such as billing, technical issues, or general queries.
     - **User Sentiment**: Evaluates the user's sentiment (positive, neutral, or negative) to determine if escalation is needed.

2. **Intelligent Routing**:
   - Based on the **query_category** and **query_sentiment**, the system routes the query to the appropriate handling node:
     - **Escalate to Human**: If the sentiment is negative, the query is escalated to a human for resolution.
     - **Generate Billing Response**: Queries related to billing are routed to generate an appropriate response.
     - **Generate Technical Response**: Technical queries are routed for a specialized technical response.
     - **Generate General Response**: General queries are handled with context-aware responses.

3. **Knowledge Base Integration (RAG) - Databricks Vector Search**:
   - The system integrates with a **Databricks Vector Search Index** to augment responses with relevant and accurate information.
   - Retrieval-Augmented Generation (RAG) ensures that responses are grounded in the latest and most reliable data.
   - Uses **Delta Sync Index** with managed embeddings (`databricks-gte-large-en`).

4. **Escalation Mechanism**:
   - Negative sentiment triggers an **escalation to a human**, ensuring the user receives empathetic and personalized support for critical issues.

### What This Notebook Covers

This notebook is **self-contained** and walks through:
1. **Loading** the knowledge base from JSON.
2. **Creating a Delta table** with the document chunks.
3. **Setting up a Databricks Vector Search Endpoint** and **Index** with managed embeddings (`databricks-gte-large-en`).
4. **Building the LangGraph Router Agent** with RAG powered by Databricks Vector Search.
5. **Testing** the full workflow with sample customer queries.

## Install Dependencies

In [ ]:
!pip install -q langchain==0.3.14
!pip install -q langchain-openai==0.3.0
!pip install -q langchain-community==0.3.14
!pip install -q langgraph==0.2.64
!pip install -q databricks-vectorsearch
!pip install -U -qqqq databricks-langchain
!pip install -q jq

In [ ]:
dbutils.library.restartPython()

## Step 1: Load the Company Knowledge Base

Load the customer support knowledge base from the JSON file. Each document contains a `text` field and `metadata` with a `category` (technical, billing, or general).

In [ ]:
import json

with open("./docs/router_agent_documents.json", "r") as f:
    knowledge_base = json.load(f)

print(f"Loaded {len(knowledge_base)} documents from knowledge base.")
knowledge_base[:3]

## Step 2: Create a Delta Table with Document Chunks

We convert the knowledge base documents into a Spark DataFrame and save it as a **Delta table** in Unity Catalog. This Delta table will serve as the source for the Vector Search index.

| Component | Value |
|---|---|
| **Catalog** | `agentic_ai` |
| **Schema** | `langgraph` |
| **Table** | `router_agent_chunks` |
| **Columns** | `chunk_id` (INT), `content` (STRING), `category` (STRING) |

In [ ]:
# Unity Catalog configuration
CATALOG = "agentic_ai"
SCHEMA = "langgraph"
TABLE_NAME = f"{CATALOG}.{SCHEMA}.router_agent_chunks"

# Convert knowledge base to chunk data
chunk_data = []
for i, doc in enumerate(knowledge_base):
    chunk_data.append({
        "chunk_id": i + 1,
        "content": doc["text"],
        "category": doc["metadata"]["category"]
    })

print(f"Created {len(chunk_data)} chunks.")
chunk_data[:3]

In [ ]:
# Create Spark DataFrame and save as Delta table
spark_df = spark.createDataFrame(chunk_data)

spark_df.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable(TABLE_NAME)

print(f"Delta table '{TABLE_NAME}' created successfully.")
display(spark.table(TABLE_NAME))

In [ ]:
# Enable Change Data Feed (required for Delta Sync Index)
spark.sql(f"ALTER TABLE {TABLE_NAME} SET TBLPROPERTIES (delta.enableChangeDataFeed = true)")
print("Change Data Feed enabled on the Delta table.")

## Step 3: Create a Vector Search Endpoint

A Vector Search **endpoint** is a compute resource that serves the vector search index. We create a `STANDARD` endpoint named `router_agent_endpoint`.

> **Note**: If the endpoint already exists, this cell will raise an error — you can safely skip it.

In [ ]:
from databricks.vector_search.client import VectorSearchClient

ENDPOINT_NAME = "router_agent_endpoint"

vs_client = VectorSearchClient()

# Create the Vector Search endpoint (skip if it already exists)
try:
    vs_client.create_endpoint(
        name=ENDPOINT_NAME,
        endpoint_type="STANDARD"
    )
    print(f"Vector Search endpoint '{ENDPOINT_NAME}' created. It may take a few minutes to provision.")
except Exception as e:
    if "already exists" in str(e).lower():
        print(f"Endpoint '{ENDPOINT_NAME}' already exists. Skipping creation.")
    else:
        raise e

## Step 4: Create the Vector Search Index (with Managed Embeddings)

We create a **Delta Sync Index** on the Delta table. This index:
- Uses **`databricks-gte-large-en`** as the managed embedding model — Databricks automatically computes embeddings from the `content` column.
- Syncs automatically with the source Delta table via `TRIGGERED` pipeline.
- Uses `chunk_id` as the primary key.

> **Note**: If the index already exists, this cell will raise an error — you can safely skip it. The index creation may take several minutes to complete.

In [ ]:
INDEX_NAME = f"{CATALOG}.{SCHEMA}.router_agent_index"

# Create the Delta Sync Index with managed embeddings
try:
    index = vs_client.create_delta_sync_index(
        endpoint_name=ENDPOINT_NAME,
        source_table_name=TABLE_NAME,
        index_name=INDEX_NAME,
        pipeline_type="TRIGGERED",
        primary_key="chunk_id",
        embedding_source_column="content",
        embedding_model_endpoint_name="databricks-gte-large-en"
    )
    print(f"Vector Search index '{INDEX_NAME}' created successfully.")
    print("Note: Embedding computation may take a few minutes. Wait for the index to become ONLINE before querying.")
except Exception as e:
    if "already exists" in str(e).lower():
        print(f"Index '{INDEX_NAME}' already exists. Skipping creation.")
    else:
        raise e

In [ ]:
import time

# Wait for the index to become ONLINE
vs_index = vs_client.get_index(endpoint_name=ENDPOINT_NAME, index_name=INDEX_NAME)

status = vs_index.describe()
print(f"Index status: {status.get('status', {})}")

# Poll until the index is ready (optional - you can also check manually in the Databricks UI)
while status.get("status", {}).get("ready") != True:
    print("Index is not ready yet. Waiting 30 seconds...")
    time.sleep(30)
    status = vs_index.describe()
    print(f"Index status: {status.get('status', {})}")

print("Index is ONLINE and ready for queries!")

## Setup Environment Variables & LLM

We use **Databricks Foundation Model APIs** via `ChatDatabricks` so no external API keys are needed.

In [ ]:
from databricks_langchain import ChatDatabricks

# Use a Databricks-hosted LLM endpoint
# Options: "databricks-claude-3-7-sonnet", "databricks-gpt-oss-120b", "databricks-meta-llama-3-3-70b-instruct"
LLM_ENDPOINT_NAME = "databricks-claude-3-7-sonnet"
llm = ChatDatabricks(endpoint=LLM_ENDPOINT_NAME)

## Step 5: Connect to the Vector Search Index

Now we connect to the Vector Search index we just created (or the pre-existing one) to use it for retrieval in our RAG pipeline.

In [ ]:
# Get the vector search index (uses ENDPOINT_NAME and INDEX_NAME defined earlier)
vs_index = vs_client.get_index(endpoint_name=ENDPOINT_NAME, index_name=INDEX_NAME)
print(f"Connected to Vector Search Index: {INDEX_NAME}")
print(f"Endpoint: {ENDPOINT_NAME}")
print(f"Source Table: {TABLE_NAME}")

## Helper: Convert Vector Search Results to LangChain Documents

Databricks Vector Search returns results in its own format. This helper converts them to LangChain `Document` objects for seamless integration with our RAG pipeline.

In [ ]:
from langchain_core.documents import Document
from typing import List

def convert_vector_search_to_documents(results) -> List[Document]:
    """
    Convert Databricks Vector Search results to LangChain Document objects.
    
    The first column retrieved is loaded into page_content,
    and the rest (except the score column) into metadata.
    """
    column_names = [col["name"] for col in results["manifest"]["columns"]]
    
    langchain_docs = []
    for item in results["result"]["data_array"]:
        metadata = {}
        # Last element is the similarity score
        score = item[-1]
        # First element is page_content, middle elements are metadata
        for i in range(1, len(item) - 1):
            metadata[column_names[i]] = item[i]
        metadata["score"] = score
        doc = Document(page_content=item[0], metadata=metadata)
        langchain_docs.append(doc)
    
    return langchain_docs

## Test Vector Search Retrieval

Let's verify that the vector search index is working correctly with some sample queries and metadata filters.

In [ ]:
# Test: Query with 'general' category filter
query = 'what is your refund policy?'
results = vs_index.similarity_search(
    query_text=query,
    columns=["content", "category"],
    num_results=3,
    filters={"category": ["general"]},
    query_type="hybrid"
)

docs = convert_vector_search_to_documents(results)
for doc in docs:
    print(f"Score: {doc.metadata['score']:.4f} | Category: {doc.metadata['category']}")
    print(f"Content: {doc.page_content[:150]}...")
    print("---")

In [ ]:
# Test: Query with 'technical' category filter
query = 'do you support pre-trained models?'
results = vs_index.similarity_search(
    query_text=query,
    columns=["content", "category"],
    num_results=3,
    filters={"category": ["technical"]},
    query_type="hybrid"
)

docs = convert_vector_search_to_documents(results)
for doc in docs:
    print(f"Score: {doc.metadata['score']:.4f} | Category: {doc.metadata['category']}")
    print(f"Content: {doc.page_content[:150]}...")
    print("---")

In [ ]:
# Test: Query with 'billing' category filter
query = 'what payment methods do you accept?'
results = vs_index.similarity_search(
    query_text=query,
    columns=["content", "category"],
    num_results=3,
    filters={"category": ["billing"]},
    query_type="hybrid"
)

docs = convert_vector_search_to_documents(results)
for doc in docs:
    print(f"Score: {doc.metadata['score']:.4f} | Category: {doc.metadata['category']}")
    print(f"Content: {doc.page_content[:150]}...")
    print("---")

## Define the Customer Inquiry State

We create a `CustomerSupportState` typed dictionary to keep track of each interaction:
- **customer_query**: The text of the customer's question
- **query_category**: Technical, Billing, or General (used for routing)
- **query_sentiment**: Positive, Neutral, or Negative (used for routing)
- **final_response**: The system's response to the customer

In [ ]:
from typing import TypedDict, Literal
from pydantic import BaseModel

class CustomerSupportState(TypedDict):
    """
    customer_query: the original query from the customer.
    query_category: the topic of the query (e.g., Technical, Billing).
    query_sentiment: the emotional tone (e.g., Positive, Negative).
    final_response: the system-generated response.
    """
    customer_query: str
    query_category: str
    query_sentiment: str
    final_response: str

class QueryCategory(BaseModel):
    categorized_topic: Literal['Technical', 'Billing', 'General']

class QuerySentiment(BaseModel):
    sentiment: Literal['Positive', 'Neutral', 'Negative']

### Explain TypeDict, Literal and BaseModel
* **TypedDict**: Allows you to define a dictionary with a specific schema (i.e., typed keys and values). This is a lightweight structure used for state management in memory (not validated like Pydantic models).
* **Literal**: Restricts a value to a fixed set of string options.
* **BaseModel** from pydantic: Used for data validation and parsing using Python type hints.

In [ ]:
QueryCategory(categorized_topic='Billing')  # ✅ Valid

In [ ]:
# QueryCategory(categorized_topic='billing')    # ❌ Validation error

## Create Node Functions

Each function below represents a stage in processing a customer inquiry:

1. **categorize_inquiry**: Classifies the query into Technical, Billing, or General.
2. **analyze_inquiry_sentiment**: Determines if the sentiment is Positive, Neutral, or Negative.
3. **generate_technical_response**: Produces a response for technical issues using **Databricks Vector Search**.
4. **generate_billing_response**: Produces a response for billing questions using **Databricks Vector Search**.
5. **generate_general_response**: Produces a response for general queries using **Databricks Vector Search**.
6. **escalate_to_human_agent**: Escalates the query to a human if sentiment is negative.
7. **determine_route**: Routes the inquiry to the appropriate response node based on category and sentiment.

In [ ]:
def categorize_inquiry(support_state: CustomerSupportState) -> CustomerSupportState:
    """
    Classify the customer query into Technical, Billing, or General.
    """

    query = support_state["customer_query"]
    ROUTE_CATEGORY_PROMPT = """Act as a customer support agent trying to best categorize the customer query.
                               You are an agent for an AI products and hardware company.

                               Please read the customer query below and
                               determine the best category from the following list:

                               'Technical', 'Billing', or 'General'.

                               Remember:
                                - Technical queries will focus more on technical aspects like AI models, hardware, software related queries etc.
                                - General queries will focus more on general aspects like contacting support, finding things, policies etc.
                                - Billing queries will focus more on payment and purchase related aspects

                                Return just the category name (from one of the above)

                                Query:
                                {customer_query}
                            """
    prompt = ROUTE_CATEGORY_PROMPT.format(customer_query=query)
    route_category = llm.with_structured_output(QueryCategory).invoke(prompt)

    return {
        "query_category": route_category.categorized_topic
    }

In [ ]:
categorize_inquiry({"customer_query": "Do you provide pretrained models?"})

In [ ]:
categorize_inquiry({"customer_query": "what is your refund policy?"})

In [ ]:
categorize_inquiry({"customer_query": "what payment methods are accepted?"})

In [ ]:
def analyze_inquiry_sentiment(support_state: CustomerSupportState) -> CustomerSupportState:
    """
    Analyze the sentiment of the customer query as Positive, Neutral, or Negative.
    """

    query = support_state["customer_query"]
    SENTIMENT_CATEGORY_PROMPT = """Act as a customer support agent trying to best categorize the customer query's sentiment.
                                   You are an agent for an AI products and hardware company.

                                   Please read the customer query below,
                                   analyze its sentiment which should be one from the following list:

                                   'Positive', 'Neutral', or 'Negative'.

                                   Return just the sentiment (from one of the above)

                                   Query:
                                   {customer_query}
                                """
    prompt = SENTIMENT_CATEGORY_PROMPT.format(customer_query=query)
    sentiment_category = llm.with_structured_output(QuerySentiment).invoke(prompt)

    return {
        "query_sentiment": sentiment_category.sentiment
    }

In [ ]:
analyze_inquiry_sentiment({"customer_query": "what is your refund policy?"})

In [ ]:
analyze_inquiry_sentiment({"customer_query": "what is your refund policy? I am really fed up with this product and need to refund it"})

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

def generate_technical_response(support_state: CustomerSupportState) -> CustomerSupportState:
    """
    Provide a technical support response by combining knowledge from Databricks Vector Search and LLM.
    """
    categorized_topic = support_state["query_category"]
    query = support_state["customer_query"]

    if categorized_topic.lower() == "technical":
        # Perform retrieval from Databricks Vector Search with category filter
        results = vs_index.similarity_search(
            query_text=query,
            columns=["content", "category"],
            num_results=3,
            filters={"category": ["technical"]},
            query_type="hybrid"
        )
        relevant_docs = convert_vector_search_to_documents(results)
        retrieved_content = "\n\n".join(doc.page_content for doc in relevant_docs)

        prompt = ChatPromptTemplate.from_template(
            """
            Craft a clear and detailed technical support response for the following customer query.
            Use the provided knowledge base information to enrich your response.
            In case there is no knowledge base information or you do not know the answer just say:

            Apologies I was not able to answer your question, please reach out to +1-xxx-xxxx

            Customer Query:
            {customer_query}

            Relevant Knowledge Base Information:
            {retrieved_content}
            """
        )

        chain = prompt | llm
        tech_reply = chain.invoke({
            "customer_query": query,
            "retrieved_content": retrieved_content
        }).content
    else:
        tech_reply = "Apologies I was not able to answer your question, please reach out to +1-xxx-xxxx"

    return {
        "final_response": tech_reply
    }

In [ ]:
generate_technical_response({"customer_query": "what is your refund policy?", "query_category": "General"})

In [ ]:
generate_technical_response({"customer_query": "do you support on-prem models?", "query_category": "Technical"})

In [ ]:
def generate_billing_response(support_state: CustomerSupportState) -> CustomerSupportState:
    """
    Provide a billing support response by combining knowledge from Databricks Vector Search and LLM.
    """
    categorized_topic = support_state["query_category"]
    query = support_state["customer_query"]

    if categorized_topic.lower() == "billing":
        # Perform retrieval from Databricks Vector Search with category filter
        results = vs_index.similarity_search(
            query_text=query,
            columns=["content", "category"],
            num_results=3,
            filters={"category": ["billing"]},
            query_type="hybrid"
        )
        relevant_docs = convert_vector_search_to_documents(results)
        retrieved_content = "\n\n".join(doc.page_content for doc in relevant_docs)

        prompt = ChatPromptTemplate.from_template(
            """
            Craft a clear and detailed billing support response for the following customer query.
            Use the provided knowledge base information to enrich your response.
            In case there is no knowledge base information or you do not know the answer just say:

            Apologies I was not able to answer your question, please reach out to +1-xxx-xxxx

            Customer Query:
            {customer_query}

            Relevant Knowledge Base Information:
            {retrieved_content}
            """
        )

        chain = prompt | llm
        billing_reply = chain.invoke({
            "customer_query": query,
            "retrieved_content": retrieved_content
        }).content
    else:
        billing_reply = "Apologies I was not able to answer your question, please reach out to +1-xxx-xxxx"

    return {
        "final_response": billing_reply
    }

In [ ]:
generate_billing_response({"customer_query": "what payment methods are supported?", "query_category": "Billing"})

In [ ]:
def generate_general_response(support_state: CustomerSupportState) -> CustomerSupportState:
    """
    Provide a general support response by combining knowledge from Databricks Vector Search and LLM.
    """
    categorized_topic = support_state["query_category"]
    query = support_state["customer_query"]

    if categorized_topic.lower() == "general":
        # Perform retrieval from Databricks Vector Search with category filter
        results = vs_index.similarity_search(
            query_text=query,
            columns=["content", "category"],
            num_results=3,
            filters={"category": ["general"]},
            query_type="hybrid"
        )
        relevant_docs = convert_vector_search_to_documents(results)
        retrieved_content = "\n\n".join(doc.page_content for doc in relevant_docs)

        prompt = ChatPromptTemplate.from_template(
            """
            Craft a clear and detailed general support response for the following customer query.
            Use the provided knowledge base information to enrich your response.
            In case there is no knowledge base information or you do not know the answer just say:

            Apologies I was not able to answer your question, please reach out to +1-xxx-xxxx

            Customer Query:
            {customer_query}

            Relevant Knowledge Base Information:
            {retrieved_content}
            """
        )

        chain = prompt | llm
        general_reply = chain.invoke({
            "customer_query": query,
            "retrieved_content": retrieved_content
        }).content
    else:
        general_reply = "Apologies I was not able to answer your question, please reach out to +1-xxx-xxxx"

    return {
        "final_response": general_reply
    }

In [ ]:
generate_general_response({"customer_query": "what is your refund policy?", "query_category": "General"})

In [ ]:
def escalate_to_human_agent(support_state: CustomerSupportState) -> CustomerSupportState:
    """
    Escalate the query to a human agent if sentiment is negative.
    """

    return {
        "final_response": "Apologies, we are really sorry! Someone from our team will be reaching out to your shortly!"
    }

In [ ]:
def determine_route(support_state: CustomerSupportState) -> str:
    """
    Route the inquiry based on sentiment and category.
    """
    if support_state["query_sentiment"] == "Negative":
        return "escalate_to_human_agent"
    elif support_state["query_category"] == "Technical":
        return "generate_technical_response"
    elif support_state["query_category"] == "Billing":
        return "generate_billing_response"
    else:
        return "generate_general_response"

## Build and Compile the Workflow

We construct a LangGraph workflow with the nodes defined above:
1. **categorize_inquiry** → **analyze_inquiry_sentiment** → **route** to the proper response node.
2. If negative, escalate to a human agent.
3. Otherwise, produce an appropriate response (technical, billing, or general) using **Databricks Vector Search** for retrieval.

In [ ]:
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

# Create the graph with our typed state
customer_support_graph = StateGraph(CustomerSupportState)

# Add nodes for each function
customer_support_graph.add_node("categorize_inquiry", categorize_inquiry)
customer_support_graph.add_node("analyze_inquiry_sentiment", analyze_inquiry_sentiment)
customer_support_graph.add_node("generate_technical_response", generate_technical_response)
customer_support_graph.add_node("generate_billing_response", generate_billing_response)
customer_support_graph.add_node("generate_general_response", generate_general_response)
customer_support_graph.add_node("escalate_to_human_agent", escalate_to_human_agent)

# Add edges to represent the processing flow
customer_support_graph.add_edge("categorize_inquiry", "analyze_inquiry_sentiment")
customer_support_graph.add_conditional_edges(
    "analyze_inquiry_sentiment",
    determine_route,
    [
        "generate_technical_response",
        "generate_billing_response",
        "generate_general_response",
        "escalate_to_human_agent"
    ]
)

# All terminal nodes lead to the END
customer_support_graph.add_edge("generate_technical_response", END)
customer_support_graph.add_edge("generate_billing_response", END)
customer_support_graph.add_edge("generate_general_response", END)
customer_support_graph.add_edge("escalate_to_human_agent", END)

# Set the entry point for the workflow
customer_support_graph.set_entry_point("categorize_inquiry")

# Compile the graph into a runnable agent
memory = MemorySaver()
compiled_support_agent = customer_support_graph.compile(checkpointer=memory)

## Visualize the Workflow

Below is a generated diagram of the workflow using Mermaid syntax. It shows how each node connects in the graph.

In [ ]:
from IPython.display import display, Image, Markdown

display(Image(compiled_support_agent.get_graph().draw_mermaid_png()))

## Helper Function to Run the Workflow

This function takes a customer query and runs it through our compiled workflow, returning the final results (category, sentiment, and generated response).

In [ ]:
def call_support_agent(agent, prompt, user_session_id, verbose=False):
    events = agent.stream(
        {"customer_query": prompt},  # initial state of the agent
        {"configurable": {"thread_id": user_session_id}},
        stream_mode="values",
    )

    print('Running Agent. Please wait...')
    for event in events:
        if verbose:
                print(event)

    display(Markdown(event['final_response']))

## Testing the Customer Support Workflow

Let's test the workflow with some sample queries to verify categorization, sentiment analysis, and response generation using **Databricks Vector Search** as the knowledge base.

In [ ]:
uid = 'jim001'
query = "do you support pre-trained models?"
call_support_agent(agent=compiled_support_agent,
                   prompt=query,
                   user_session_id=uid,
                   verbose=True)

In [ ]:
uid = 'jim002'
query = "how do I get my invoice?"
call_support_agent(agent=compiled_support_agent,
                   prompt=query,
                   user_session_id=uid,
                   verbose=True)

In [ ]:
query = "Can you tell me about your shipping policy?"
call_support_agent(agent=compiled_support_agent,
                   prompt=query,
                   user_session_id=uid,
                   verbose=False)

In [ ]:
query = "I'm fed up with this faulty hardware, I need a refund"
call_support_agent(agent=compiled_support_agent,
                   prompt=query,
                   user_session_id=uid,
                   verbose=True)

In [ ]:
query = "What are your working hours?"
call_support_agent(agent=compiled_support_agent,
                   prompt=query,
                   user_session_id=uid,
                   verbose=True)

In [ ]:
query = "What have I asked you till now"
call_support_agent(agent=compiled_support_agent,
                   prompt=query,
                   user_session_id=uid,
                   verbose=True)